In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [3]:
df = pd.read_csv("churndata.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:

df_x = df.iloc[:, 3:13]
df_y = df.iloc[:, 13]

# df.isna().sum()

def clean_data(df):

    le = LabelEncoder()
    df.Gender = le.fit_transform(df.Gender)
    df = pd.get_dummies(data = df, columns=["Geography"], drop_first = False)
    df = df.sort_index(axis=1)
    return df

df_x = clean_data(df_x)
# df_x.head()

# columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# col_tnf = columnTransformer.fit_transform(df_x)
# df_x = np.array(col_tnf, dtype = np.str)
# df_x = df_x[:, 1:]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 0)
#joblib.dump(X_train.columns, "columns.pkl")

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#joblib.dump(scaler, "std_scaler.pkl")
print(X_test)
print(X_train.shape[1])


[[-0.36890377  0.8793029  -0.55204276 ...  0.9687384  -0.92159124
   1.04473698]
 [ 0.10961719  0.42972196 -1.31490297 ... -1.03227043 -0.92159124
  -1.031415  ]
 [ 0.30102557  0.30858264  0.57162971 ...  0.9687384  -0.92159124
   1.04473698]
 ...
 [-0.27319958  1.29745526 -0.74791227 ... -1.03227043  0.8095029
  -1.37744033]
 [-0.46460796  1.05975239 -0.00566991 ...  0.9687384  -0.92159124
  -0.33936434]
 [-0.84742473  0.82026342 -0.79945688 ... -1.03227043 -0.92159124
   1.04473698]]
12


In [6]:
# Build basic ANN model 
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 30)
generate_report()

Epoch 1/30
800/800 [==============================] - 5s 5ms/step - loss: 0.6133 - accuracy: 0.7949
Epoch 2/30
800/800 [==============================] - 2s 3ms/step - loss: 0.5335 - accuracy: 0.7960
Epoch 3/30
800/800 [==============================] - 3s 3ms/step - loss: 0.5119 - accuracy: 0.7960
Epoch 4/30
800/800 [==============================] - 2s 3ms/step - loss: 0.5070 - accuracy: 0.7960
Epoch 5/30
800/800 [==============================] - 4s 5ms/step - loss: 0.5061 - accuracy: 0.7960
Epoch 6/30
800/800 [==============================] - 3s 3ms/step - loss: 0.5060 - accuracy: 0.7960
Epoch 7/30
800/800 [==============================] - 2s 3ms/step - loss: 0.5060 - accuracy: 0.7960
Epoch 8/30
800/800 [==============================] - 2s 3ms/step - loss: 0.5059 - accuracy: 0.7960
Epoch 9/30
800/800 [==============================] - 2s 3ms/step - loss: 0.5059 - accuracy: 0.7960
Epoch 10/30
800/800 [==============================] - 3s 3ms/step - loss: 0.5059 - accuracy: 0.7960

NameError: name 'generate_report' is not defined

In [8]:
generate_report()

63/63 [==============================] - 0s 3ms/step

Predicted values: [[0.20611185]
 [0.20611185]
 [0.20611185]
 ...
 [0.20611185]
 [0.20611185]
 [0.20611185]]


Test Accuracy: 0.7975



In [9]:

# Implementing K-fold Cross validation

def build_model():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_model, batch_size = 10, epochs = 40)
accuracy_list = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
mean_acc = accuracy_list.mean()
std_acc = accuracy_list.std()
print(mean_acc, std_acc)
classifier.fit(X_train, y_train)
generate_report()


Epoch 1/40


C:\Users\52296881\AppData\Local\Temp/ipykernel_848/690373569.py:10: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_model, batch_size = 10, epochs = 40)


720/720 [==============================] - 4s 3ms/step - loss: 0.5028 - accuracy: 0.7974
Epoch 2/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4198 - accuracy: 0.8218
Epoch 3/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4134 - accuracy: 0.8279
Epoch 4/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4086 - accuracy: 0.8297
Epoch 5/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4051 - accuracy: 0.8324
Epoch 6/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4021 - accuracy: 0.8336
Epoch 7/40
720/720 [==============================] - 3s 5ms/step - loss: 0.3976 - accuracy: 0.8347
Epoch 8/40
720/720 [==============================] - 3s 5ms/step - loss: 0.3923 - accuracy: 0.8372
Epoch 9/40
720/720 [==============================] - 2s 3ms/step - loss: 0.3835 - accuracy: 0.8400
Epoch 10/40
720/720 [==============================] - 2s 3ms/step - loss: 0.3773 - accuracy: 0.8418
Epoch 11/4

80/80 [==============================] - 1s 4ms/step - loss: 0.3999 - accuracy: 0.8388
Epoch 1/40
720/720 [==============================] - 4s 5ms/step - loss: 0.4902 - accuracy: 0.7950
Epoch 2/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4307 - accuracy: 0.7956
Epoch 3/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4258 - accuracy: 0.7956
Epoch 4/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4217 - accuracy: 0.8101
Epoch 5/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4190 - accuracy: 0.8243
Epoch 6/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4167 - accuracy: 0.8286
Epoch 7/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4150 - accuracy: 0.8301
Epoch 8/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4134 - accuracy: 0.8311
Epoch 9/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4127 - accuracy: 0.8326
Epoch 10/40
7

80/80 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8313
Epoch 1/40
720/720 [==============================] - 3s 2ms/step - loss: 0.5022 - accuracy: 0.7928
Epoch 2/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4342 - accuracy: 0.7937
Epoch 3/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4299 - accuracy: 0.7937
Epoch 4/40
720/720 [==============================] - 3s 5ms/step - loss: 0.4257 - accuracy: 0.7961
Epoch 5/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4224 - accuracy: 0.8189
Epoch 6/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4198 - accuracy: 0.8229
Epoch 7/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4182 - accuracy: 0.8268
Epoch 8/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4167 - accuracy: 0.8268
Epoch 9/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4156 - accuracy: 0.8278
Epoch 10/40
7

720/720 [==============================] - 3s 4ms/step - loss: 0.4012 - accuracy: 0.8338
Epoch 40/40
80/80 [==============================] - 0s 3ms/step - loss: 0.4069 - accuracy: 0.8363
Epoch 1/40
720/720 [==============================] - 3s 3ms/step - loss: 0.4933 - accuracy: 0.7968
Epoch 2/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4312 - accuracy: 0.7969
Epoch 3/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4272 - accuracy: 0.7969
Epoch 4/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4240 - accuracy: 0.7968
Epoch 5/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4218 - accuracy: 0.8150
Epoch 6/40
720/720 [==============================] - 3s 3ms/step - loss: 0.4185 - accuracy: 0.8231
Epoch 7/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4163 - accuracy: 0.8251
Epoch 8/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4145 - accuracy: 0.8272
Epoch 9/40
7

720/720 [==============================] - 2s 3ms/step - loss: 0.3995 - accuracy: 0.8356
Epoch 39/40
720/720 [==============================] - 2s 3ms/step - loss: 0.3989 - accuracy: 0.8386
Epoch 40/40
80/80 [==============================] - 0s 3ms/step - loss: 0.4195 - accuracy: 0.8263
Epoch 1/40
720/720 [==============================] - 4s 5ms/step - loss: 0.4965 - accuracy: 0.7953
Epoch 2/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4273 - accuracy: 0.7957
Epoch 3/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4217 - accuracy: 0.8001
Epoch 4/40
720/720 [==============================] - 3s 4ms/step - loss: 0.4162 - accuracy: 0.8242
Epoch 5/40
720/720 [==============================] - 3s 5ms/step - loss: 0.4111 - accuracy: 0.8336
Epoch 6/40
720/720 [==============================] - 2s 3ms/step - loss: 0.4067 - accuracy: 0.8336
Epoch 7/40
720/720 [==============================] - 2s 2ms/step - loss: 0.4037 - accuracy: 0.8350
Epoch 8/40


720/720 [==============================] - 2s 2ms/step - loss: 0.3458 - accuracy: 0.8639
Epoch 38/40
720/720 [==============================] - 1s 2ms/step - loss: 0.3454 - accuracy: 0.8585
Epoch 39/40
720/720 [==============================] - 2s 3ms/step - loss: 0.3463 - accuracy: 0.8608
Epoch 40/40
80/80 [==============================] - 1s 4ms/step - loss: 0.3292 - accuracy: 0.8725
0.8396250009536743 0.013670796740935423
Epoch 1/40
800/800 [==============================] - 4s 4ms/step - loss: 0.4838 - accuracy: 0.7955
Epoch 2/40
800/800 [==============================] - 4s 5ms/step - loss: 0.4248 - accuracy: 0.7984
Epoch 3/40
800/800 [==============================] - 3s 3ms/step - loss: 0.4168 - accuracy: 0.8234
Epoch 4/40
800/800 [==============================] - 2s 2ms/step - loss: 0.4110 - accuracy: 0.8314
Epoch 5/40
800/800 [==============================] - 2s 3ms/step - loss: 0.4077 - accuracy: 0.8322
Epoch 6/40
800/800 [==============================] - 2s 3ms/step - lo

In [11]:

# implement Hyperparameter tuning for getting better accuracy using Grid Search
def build_model(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_model)
parameters = {"batch_size": [5, 15, 30], 
              "epochs": [30, 100],
              "optimizer": ["adam", "rmsprop"]}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = "accuracy",
                           cv=10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_parameters, best_accuracy)
generate_report()

SyntaxError: unterminated string literal (detected at line 4) (Temp/ipykernel_848/1213240838.py, line 4)

In [7]:
def generate_report():
    # predicting values
    y_pred = classifier.predict(X_test)
    print("\nPredicted values: "+str(y_pred)+"\n")
    y_pred = (y_pred > 0.5)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    accuracy = (cm[0,0]+cm[1, 1])/(cm[0,0]+cm[1, 1]+cm[1,0]+cm[0, 1])
    print("\nTest Accuracy: "+str(accuracy)+"\n")

In [ ]:

# save the model so created above into a picle.
joblib.dump(classifier, 'prediction_classifier.pkl') 